### Part 4 Clustering and Association Mining to Web Data

In [3]:
import pandas as pd

#### Import cleaned dataset

In [7]:
df = pd.read_csv('CaseStudy2Data\web_log_data.csv' )
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5866 entries, 0 to 5865
Data columns (total 6 columns):
ip           5866 non-null object
date_time    5866 non-null object
request      5866 non-null object
step         5866 non-null int64
session      5866 non-null int64
user_id      5866 non-null int64
dtypes: int64(3), object(3)
memory usage: 275.0+ KB


,ip,date_time,request,step,session,user_id
0,c210-49-32-6.rochd2.,18/Apr/2005:21:25:07,/,1,3,3
1,visp.inabox.telstra.,19/Apr/2005:08:24:28,/,1,12,12
2,dsl-61-95-54-84.requ,19/Apr/2005:08:33:01,/,1,13,13
3,d220-236-91-52.dsl.n,19/Apr/2005:09:16:06,/,1,15,15
4,allptrs.eq.edu.au,19/Apr/2005:09:47:54,/,1,22,22
5,cpe-144-136-135-38.q,19/Apr/2005:10:13:37,/,1,23,23
6,225-145-222-203.rev.,19/Apr/2005:11:48:32,/,1,25,25
7,cpe-138-130-198-54.q,19/Apr/2005:12:31:54,/,1,26,26
8,203-219-44-170-qld.t,19/Apr/2005:12:33:49,/,1,29,29
9,cpe-138-130-198-54.q,19/Apr/2005:12:42:51,/,1,30,30


In [22]:
#print(df['user_id'].value_counts())

In [23]:
#df = pd.concat([df,pd.get_dummies(df['ip'], prefix='ip', prefix_sep='_', columns= (''))], axis=1)
#df.drop(['ip'],axis=1, inplace=True)

In [8]:
from sklearn.preprocessing import StandardScaler

# take 3 variables and drop the rest
df2 = df[['step']]

# convert df2 to matrix
X = df2.as_matrix()

# scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

D:\Programs\conda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
D:\Programs\conda\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
D:\Programs\conda\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [9]:
from sklearn.cluster import KMeans

# random state, we will use 42 instead of 10 for a change
rs = 42

# set the random state. different random state seeds might result in different centroids locations
model = KMeans(n_clusters=3, random_state=rs)
model.fit(X)

# sum of intra-cluster distances
print("Sum of intra-cluster distance:", model.inertia_)

print("Centroid locations:")
for centroid in model.cluster_centers_:
    print(centroid)

Sum of intra-cluster distance: 1199.006381150884
Centroid locations:
[-0.40608193]
[3.4902619]
[0.89280273]


In [10]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram
import seaborn as sns

model = KMeans(n_clusters=3, random_state=rs).fit(X)

# assign cluster ID to each record in X
# Ignore the warning, does not apply to our case here
y = model.predict(X)
df2['Cluster_ID'] = y

# how many records are in each cluster
print("Cluster membership")
print(df2['Cluster_ID'].value_counts())

# pairplot the cluster distribution.
cluster_g = sns.pairplot(df2, hue='Cluster_ID')
plt.show()

D:\Programs\conda\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


Cluster membership
0    4534
2    1081
1     251
Name: Cluster_ID, dtype: int64


D:\Programs\conda\lib\site-packages\statsmodels\nonparametric\kde.py:488: RuntimeWarning: invalid value encountered in true_divide
  binned = fast_linbin(X, a, b, gridsize) / (delta * nobs)
D:\Programs\conda\lib\site-packages\statsmodels\nonparametric\kdetools.py:34: RuntimeWarning: invalid value encountered in double_scalars
  FAC1 = 2*(np.pi*bw/RANGE)**2


<Figure size 582.111x500 with 6 Axes>

#### Analysis
step
* Cluster 0 have considerable higher number of steps taken during surfing
* Cluster 1 and 2, both have minimal steps taken (0-20) in a single log in

session
* Cluster 0 have dense distributed session id and significantlyhave more steps (>20)  
* Cluster 1 and have session id 1-1000, 2000 respectively. Both have only steps of 20 or less in each session.

In [ ]:
print(df['step'].value_counts())

### Part4.2 Association mining

In [15]:
# group by iphost, then list all requestlinks
links = df.groupby(['ip'])['request'].apply(list)
print(links.head(20))

ip
002.b.004.brs.iprimu                                  [/, /services.html]
034.mel0205.mel.ipri                                                  [/]
038.b.004.brs.iprimu                                                  [/]
042.058.dsl.mel.ipri                                                  [/]
058.a.001.cns.iprimu    [/eaglefarm/, /eaglefarm/javascript/menu.js, /...
081.a.004.syd.iprimu                                                  [/]
086.a.002.brs.iprimu         [/eaglefarm/, /eaglefarm/javascript/menu.js]
099.a.001.brs.iprimu                             [/, /, /springwood.html]
1.cust21.qld.dsl.oze                                    [/, /favicon.ico]
103.cust20.qld.dsl.o    [/, /favicon.ico, /favicon.ico, /favicon.ico, ...
107.a.004.brs.iprimu    [/, /, /acacia.html, /direct.html, /eaglefarm,...
11.128-142-203.dart.                    [/, /services.html, /whoare.html]
114.a.001.brs.iprimu    [/, /acacia.html, /direct.html, /eaglefarm, /e...
118.a.002.gct.iprimu    [/, /acacia

In [16]:
from apyori import apriori

# type cast the transactions from pandas into normal list format and run apriori
links_list = list(links)
results = list(apriori(links_list, min_support=0.05))

# print first 5 rules
print(results[:5])

[RelationRecord(items=frozenset({'/'}), support=0.6377952755905512, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/'}), confidence=0.6377952755905512, lift=1.0)]), RelationRecord(items=frozenset({'/direct.html'}), support=0.0764904386951631, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/direct.html'}), confidence=0.0764904386951631, lift=1.0)]), RelationRecord(items=frozenset({'/eaglefarm'}), support=0.06299212598425197, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/eaglefarm'}), confidence=0.06299212598425197, lift=1.0)]), RelationRecord(items=frozenset({'/eaglefarm/'}), support=0.23397075365579303, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'/eaglefarm/'}), confidence=0.23397075365579303, lift=1.0)]), RelationRecord(items=frozenset({'/eaglefarm/contact'}), support=0.05061867266591676, ordered_statistics=[OrderedStatistic(items_base=fr

In [17]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 'Confidence', 'Lift']) 

result_df = convert_apriori_results_to_pandas_df(results)

print(result_df.head(20))

   Left_side                         Right_side   Support  Confidence  Lift
0                                             /  0.637795    0.637795   1.0
1                                  /direct.html  0.076490    0.076490   1.0
2                                    /eaglefarm  0.062992    0.062992   1.0
3                                   /eaglefarm/  0.233971    0.233971   1.0
4                            /eaglefarm/contact  0.050619    0.050619   1.0
5                        /eaglefarm/default.htm  0.068616    0.068616   1.0
6                 /eaglefarm/javascript/menu.js  0.255343    0.255343   1.0
7             /eaglefarm/pdf/Web_Price_List.pdf  0.160855    0.160855   1.0
8                          /eaglefarm/pricelist  0.173228    0.173228   1.0
9                         /eaglefarm/pricelist/  0.174353    0.174353   1.0
10                          /eaglefarm/specials  0.071991    0.071991   1.0
11                         /eaglefarm/specials/  0.098988    0.098988   1.0
12          

In [20]:
# sort all acquired rules descending by lift
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(20))

                                             Left_side  \
148                                      /,/richlands/   
268           /richlands/javascript/menu.js,/richlands   
147                                       /,/richlands   
269          /richlands/javascript/menu.js,/richlands/   
267                             /richlands/,/richlands   
115                                        /richlands/   
114                                         /richlands   
118                                        /richlands/   
119                      /richlands/javascript/menu.js   
116                                         /richlands   
117                      /richlands/javascript/menu.js   
262          /eaglefarm/specials/,/eaglefarm/pricelist   
411  /eaglefarm/pricelist/,/eaglefarm/specials/,/ea...   
485  /eaglefarm/pricelist/,/eaglefarm/specials/,/ea...   
387  /eaglefarm/specials/,/eaglefarm/javascript/men...   
403  /eaglefarm/specials/,/eaglefarm/pdf/Web_Price_...   
507  /eaglefar

#### Sequence Mining

In [35]:
transactions = df.groupby(['ip'])['request'].apply(list)
sequences = transactions.values.tolist()

# show the first 5 sequences

print(sequences[:10])

[['/', '/services.html'], ['/'], ['/'], ['/'], ['/eaglefarm/', '/eaglefarm/javascript/menu.js', '/eaglefarm/pdf/Web_Price_List.pdf', '/eaglefarm/pdf/Web_Price_List.pdf', '/eaglefarm/pdf/Web_Price_List.pdf', '/eaglefarm/pdf/Web_Price_List.pdf', '/eaglefarm/pricelist', '/eaglefarm/pricelist/'], ['/'], ['/eaglefarm/', '/eaglefarm/javascript/menu.js'], ['/', '/', '/springwood.html'], ['/', '/favicon.ico'], ['/', '/favicon.ico', '/favicon.ico', '/favicon.ico', '/favicon.ico', '/favicon.ico', '/favicon.ico', '/favicon.ico', '/favicon.ico', '/index.html', '/services.html']]


In [36]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 'seq_rule_input.txt', 'seq_rule_output.txt', '10%', '10%'], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

In [37]:
get_association_rules(sequences, 0.1, 0.1)

FileNotFoundError: [Errno 2] No such file or directory: 'seq_rule_output.txt'